In [0]:
import pandas as pd
import polars as pl
import os,sys,joblib

In [0]:
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")

print("Thread limits set.")

In [0]:
sys.path.append("/Workspace/9900-f18a-cake/mt-method2/src")
TREE_PATH = "/Workspace/9900-f18a-cake/mt-method2/data/freeze0525/diseaseTree_mapped.joblib"
tree = joblib.load(TREE_PATH)
root = getattr(tree, "root", tree)

In [0]:
def get_name(n): 
    return getattr(n, "name", getattr(n, "label", "UNKNOWN"))

In [0]:
def get_children(n):
    kids = getattr(n, "children", None)
    if kids is None:
        kids = getattr(n, "child_nodes", [])
    return kids or []

In [0]:
def list_trainable(n, out=None):
    out = out or []
    kids = get_children(n)
    if len(kids) >= 2: out.append(get_name(n))
    for k in kids: list_trainable(k, out)
    return out

In [0]:
nodes = list_trainable(root)
print("Trainable parents:", len(nodes), nodes[:10])

In [0]:
raise_on_error = True
disable_dm = True
prefilter_topk = 200    
prefilter_scan_max = 5000  
prefilter_chunk_size = 1000     
only_node = "Haematological malignancy"

os.environ.setdefault("OMP_NUM_THREADS","1")
os.environ.setdefault("OPENBLAS_NUM_THREADS","1")
os.environ.setdefault("MKL_NUM_THREADS","1")
os.environ["RF_N_JOBS"] = "1"
os.environ["CV_N_JOBS"] = "1"
os.environ["MCH_PREFILTER_TOPK"] = str(prefilter_topk)
os.environ["MCH_PREFILTER_SCAN_MAX"] = str(prefilter_scan_max)
os.environ["MCH_PREFILTER_CHUNK_SIZE"] = str(prefilter_chunk_size)

if disable_dm:
    os.environ["MCH_DISABLE_DM"] = "1"

if only_node:
    os.environ["MCH_ONLY_NODE"] = only_node

print("Environment variables set.")

In [0]:
test = pl.read_csv(
    "/Volumes/cb_prod/comp9300-9900-f18a-cake/9900-f18a-cake/data/mvalue_outputs_masked_subset_leukaemia_subsampled/MValue_polaris_pivot_0.csv"
)
print(test.height, test.width)
print(test.head(2))


In [0]:
sys.path.append(r"/Workspace/9900-f18a-cake/working_branch/src")
from mch.models.training import BatchModelTrainer

print("Starting training...")
trainer = BatchModelTrainer()
stats = trainer.train_all_models(raise_on_error=raise_on_error)

print("Training finished!")
display(stats) 